In [ ]:
import numpy as np
from pydrake.all import HPolyhedron, VPolytope, Hyperellipsoid, IrisOptions, Iris
import  matplotlib.pyplot as plt

In [ ]:
dmin = -3*np.ones(3)
dmax = 3*np.ones(3)
domain2d = HPolyhedron.MakeBox(dmin[:-1], dmax[:-1])

In [ ]:
def sorted_vertices(vpoly):
    assert vpoly.ambient_dimension() == 2
    poly_center = np.sum(vpoly.vertices(), axis=1) / vpoly.vertices().shape[1]
    vertex_vectors = vpoly.vertices() - np.expand_dims(poly_center, 1)
    sorted_index = np.arctan2(vertex_vectors[1], vertex_vectors[0]).argsort()
    return vpoly.vertices()[:, sorted_index]

def plot_hpoly_matplotlib(ax, HPoly, color = None, zorder = 0):
    v = sorted_vertices(VPolytope(HPoly)).T#s
    v = np.concatenate((v, v[0,:].reshape(1,-1)), axis=0)
    if color is None:
        p = ax.plot(v[:,0], v[:,1], linewidth = 2, alpha = 0.7, zorder = zorder)
    else:
        p = ax.plot(v[:,0], v[:,1], linewidth = 2, alpha = 0.7, c = color, zorder = zorder)

    ax.fill(v[:,0], v[:,1], alpha = 0.5, c = p[0].get_color(), zorder = zorder)


def plot_ellipse(ax, H, n_samples= 50, color = None, linewidth = 1):
    A = H.A()
    center = H.center()
    angs = np.linspace(0, 2*np.pi, n_samples+1)
    coords = np.zeros((2, n_samples + 1))
    coords[0, :] = np.cos(angs)
    coords[1, :] = np.sin(angs)
    Bu = np.linalg.inv(A)@coords
    pts = Bu + center.reshape(2,1)
    if color is None:
        ax.plot(pts[0, :], pts[1, :], linewidth = linewidth)
    else:
        ax.plot(pts[0, :], pts[1, :], linewidth = linewidth, color = color)

In [ ]:
N = 100
np.random.seed(1)
pts = 2*(0.5-np.random.rand(N,2))
CE = Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(pts.T)
region = HPolyhedron(VPolytope(pts.T))
IE = region.MaximumVolumeInscribedEllipsoid() 

fig,ax = plt.subplots()
plot_hpoly_matplotlib(ax, region)
plot_ellipse(ax, CE)
plot_ellipse(ax, IE)

In [ ]:
#3d plotting in meshcat
from pydrake.all import Rgba, TriangleSurfaceMesh, SurfaceTriangle, Meshcat, StartMeshcat
from scipy.spatial import ConvexHull
def flip_triangles(triangles, vertices, point):
    flipped_triangles = []
    for triangle in triangles:
        # Calculate normal of the triangle
        v0 = vertices[triangle[0]]
        v1 = vertices[triangle[1]]
        v2 = vertices[triangle[2]]
        normal = np.cross(v1 - v0, v2 - v0)
        
        # Calculate vector from any vertex to the point
        vector_to_point = point - v0
        
        # Check if the dot product of the normal and vector to point is negative,
        # indicating that the normal is pointing towards the point
        if np.dot(normal, vector_to_point) < 0:
            # Flip the order of vertices to reverse the normal
            flipped_triangle = [triangle[0], triangle[2], triangle[1]]
            flipped_triangles.append(flipped_triangle)
        else:
            flipped_triangles.append(triangle)
    return flipped_triangles

def plot_hpoly3d(meshcat, name, hpoly, color, wireframe = True, resolution = -1, offset = np.zeros(3)):
    #meshcat wierdness of double rendering
    hpoly = HPolyhedron(hpoly.A(), hpoly.b() + 0.05*(np.random.rand(hpoly.b().shape[0])-0.5))
    verts = VPolytope(hpoly).vertices().T
    hull = ConvexHull(verts)
    triangles = []
    for s in hull.simplices:
        triangles.append(s)
    point = np.mean(verts, axis = 0)
    tri_flip = flip_triangles(triangles,verts, point)
    tri_drake = [SurfaceTriangle(*t) for t in tri_flip]
    # obj = self[name]
    # objwf = self[name+'wf']
    # col = to_hex(color)
    #material = MeshLambertMaterial(color=col, opacity=opacity)
    color2 = Rgba(0.8*color.r(), 0.8*color.g(), 0.8*color.b(), color.a())
    meshcat.SetObject(name, TriangleSurfaceMesh(tri_drake, verts+offset.reshape(-1,3)),
                            color, wireframe=False)
    meshcat.SetObject(name+'wf', TriangleSurfaceMesh(tri_drake, verts+offset.reshape(-1,3)),
                            color2, wireframe=True)

In [ ]:
meshcat = StartMeshcat()
N = 20
#np.random.seed(1)
pts = 2*(0.5-np.random.rand(N,3))
CE = Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(pts.T)
region = HPolyhedron(VPolytope(pts.T))
IE = region.MaximumVolumeInscribedEllipsoid() 
plot_hpoly3d(meshcat, 'test', region, Rgba(1,0,0,1))

In [ ]:
from tqdm import tqdm
from typing import List

def point_in_regions(pt, regions: List[HPolyhedron]):
    for r in regions:
        if r.PointInSet(pt.reshape(-1,1)):
            return True
    return False

def check_edge(point, other, obstacles: List[HPolyhedron], n_checks = 50):
    tval = np.linspace(0, 1, n_checks)
    for t in tval:
        pt = (1-t)*point + t* other
        if point_in_regions(pt, obstacles):
            return False
    return True

def vgraph(points, obstacles: List[HPolyhedron]):
    n = len(points)
    adj_mat = np.zeros(n,n)
    for i in tqdm(range(n)):
        point = points[i, :]
        for j in range(len(points[:i])):
            other = points[j]
            if check_edge(point, other, obstacles):
                adj_mat[i,j] = adj_mat[j,i] = 1
    return adj_mat.toarray()


# computing cliques
from pydrake.all import MaxCliqueSolverViaGreedy
solver = MaxCliqueSolverViaGreedy()

solver.SolveMaxClique(...... ad matrix )